# BERT Fine-Tuning

By Chris McCormick and Nick Ryan

Le jeton `[CLS]` dans BERT (Bidirectional Encoder Representations from Transformers) sert de jeton spécial utilisé pour diverses tâches, y compris le fine-tuning. Voici comment il est généralement utilisé :

1. **Phase de pré-entraînement** :
   - Pendant la phase de pré-entraînement de BERT, étant donné 2 phrases, le modèle apprend à prédire si la 2ème phrase est la vraie phrase, qui suit la 1ère phrase. Pour cette tâche, nous avons besoin d'un  jeton [CLS], dont la sortie nous indiquera la probabilité que la phrase actuelle soit la phrase suivante de la 1ère phrase.

2. **Phase de fine-tuning** :
   - Dans la phase de fine-tuning, le modèle BERT pré-entraîné peut être adapté à des tâches spécifiques comme la classification de texte, la reconnaissance d'entités nommées, les questions-réponses, etc. L'état caché final du jeton `[CLS]` peut être utilisé comme entrée pour un classificateur spécifique à la tâche.

   - Pour les tâches de classification de texte, vous pouvez utiliser l'état caché final du jeton `[CLS]` et l'injecter dans un classificateur linéaire simple (par exemple, une seule couche dense) pour effectuer des prédictions pour la tâche spécifique.

   - Pour des tâches comme la reconnaissance d'entités nommées ou les questions-réponses, vous pourriez avoir besoin d'ajouter des couches supplémentaires et de fine-tuner le modèle davantage.

   - Essentiellement, vous exploitez la capacité du modèle BERT pré-entraîné à comprendre le contexte et l'utilisez comme extracteur de caractéristiques pour votre tâche spécifique.

   - Le processus de fine-tuning implique généralement l'entraînement des couches supplémentaires spécifiques à la tâche tout en maintenant les couches pré-entraînées de BERT gelées (ou en les fine-tunant avec un taux d'apprentissage très faible).

   - L'état caché final du jeton `[CLS]` capture des informations sur l'ensemble de la séquence d'entrée, ce qui est particulièrement utile pour les tâches qui nécessitent une compréhension du contexte global.



## Installation de la bibliothèque Hugging Face

Installons le paquet [transformers](https://github.com/huggingface/transformers) de Hugging Face qui nous donnera une interface pytorch pour travailler avec BERT (cette bibliothèque contient des interfaces pour d'autres modèles de langage pré-entraînés comme GPT et GPT-2 d'OpenAI). Nous avons choisi l'interface pytorch parce qu'elle établit un bon équilibre entre les API de haut niveau (qui sont faciles à utiliser mais ne permettent pas de comprendre comment les choses fonctionnent) et le code tensorflow (qui contient beaucoup de détails mais nous détourne souvent vers des leçons sur tensorflow, alors que l'objectif ici est BERT !)

Pour le moment, la bibliothèque Hugging Face semble être l'interface pytorch la plus largement acceptée et la plus puissante pour travailler avec BERT. En plus de supporter une variété de différents modèles de transformateurs pré-entraînés, la bibliothèque inclut également des modifications pré-construites de ces modèles adaptées à votre tâche spécifique. Par exemple, dans ce tutoriel, nous utiliserons `BertForSequenceClassification`.

La bibliothèque comprend également des classes spécifiques pour la classification des jetons, la réponse aux questions, la prédiction de la phrase suivante, etc. L'utilisation de ces classes préconstruites simplifie le processus de modification de BERT pour vos besoins.

In [1]:
!pip install transformers

##Loading CoLA Dataset


Nous utiliserons l'ensemble de données [The Corpus of Linguistic Acceptability (CoLA)] (https://nyu-mll.github.io/CoLA/) pour la classification des phrases simples. Il s'agit d'un ensemble de phrases étiquetées comme grammaticalement correctes ou incorrectes. Il a été publié pour la première fois en mai 2018, et est l'un des tests inclus dans le "GLUE Benchmark" sur lequel des modèles comme BERT sont en compétition.

Nous utiliserons le paquet `wget` pour télécharger le jeu de données sur le système de fichiers de l'instance Colab.

In [2]:
!pip install wget

L'ensemble des données est hébergé sur GitHub dans ce dossier : https://nyu-mll.github.io/CoLA/

In [3]:
import wget
import os

print('Telechargement de lensemble des donnees...')

# L'URL du fichier zip du jeu de données.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Télécharger le fichier (si ce n'est pas déjà fait)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

Décompressez l'ensemble de données sur le système de fichiers. Vous pouvez parcourir le système de fichiers de l'instance de Colab dans la barre latérale de gauche.

In [4]:
# Unzip the dataset (if we haven't already)
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

## Parse

Nous pouvons voir d'après les noms de fichiers que les versions `tokenized` et `raw` des données sont toutes deux disponibles.

Nous ne pouvons pas utiliser la version pré-renseignée car, afin d'appliquer le BERT pré-entraîné, nous *devons* utiliser le tokenizer fourni par le modèle. Ceci est dû au fait que (1) le modèle a un vocabulaire spécifique et fixe et (2) le tokenizer de l'ORET a une façon particulière de traiter les mots hors-vocabulaire.

Nous allons utiliser pandas pour analyser l'ensemble d'apprentissage "in-domain" et examiner quelques-unes de ses propriétés et points de données.

In [5]:
import pandas as pd

# Charger le jeu de données dans un dataframe pandas.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Indiquer le nombre de phrases.
print('Nombre de phrases de formation: {:,}\n'.format(df.shape[0]))

# Afficher 10 lignes aléatoires des données.
df.sample(10)

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
6604,g_81,1,NaN,"John hummed, and Mary sang, the same tune."
7723,ad03,0,*,There arrived by Medea.
8035,ad03,0,*,Can will he do it?
4682,ks08,1,NaN,These are the books that we have gone most tho...
8205,ad03,0,*,I destroyed there.
5044,ks08,0,*,I believe that the problem is not easy to be o...
5501,b_73,1,NaN,Such a scholar as you were speaking of just no...
4388,ks08,0,*,Margaret has had already left.
671,bc01,1,NaN,"Smith loaned, and his widow later donated, a v..."
6207,c_13,1,NaN,What did Jean think was likely to have been st...


Les deux propriétés qui nous intéressent sont la `sentence` et son `étiquette`, que l'on appelle le "jugement d'acceptabilité" (0=inacceptable, 1=acceptable).

Voici cinq phrases étiquetées comme n'étant pas grammaticalement acceptables. Notez à quel point cette tâche est plus difficile que l'analyse des sentiments !

In [6]:
df.loc[df.label == 0].sample(5)[['sentence', 'label']]

,sentence,label
5839,I've never seen him eats asparagus.,0
1770,Dean drank more booze than Frank ate Wheaties ...,0
3447,The professor found some strong evidences of w...,0
1287,I went to the store to have bought some whisky.,0
858,John is being discussed and Sally is being too.,0


Extrayons les phrases et les étiquettes de notre ensemble d'apprentissage sous forme de tableaux numpy ndarrays.

In [7]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values

## Tokenisation et formatage des entrées

Dans cette section, nous allons transformer notre ensemble de données dans le format sur lequel BERT peut être formé.

### BERT Tokenizer

Pour transmettre notre texte à l'ORET, il doit être divisé en tokens, puis ces tokens doivent être mis en correspondance avec leur index dans le vocabulaire du tokéniseur.

La tokenisation doit être effectuée par le tokenizer inclus dans BERT - la cellule ci-dessous le téléchargera pour nous. Nous utiliserons ici la version "non casée".

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Appliquons le tokenizer à une phrase pour voir le résultat.

In [9]:
# Imprimer la phrase originale.
print(' Original: ', sentences[0])

# Imprimer la phrase divisée en tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Imprimer la phrase associée aux identifiants des jetons.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token IDs:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]


Lorsque nous convertirons toutes nos phrases, nous utiliserons la fonction `tokenize.encode` pour gérer les deux étapes, plutôt que d'appeler `tokenize` et `convert_tokens_to_ids` séparément.

Mais avant cela, nous devons parler des exigences de formatage de BERT.

### Formatage requis

Le code ci-dessus a omis quelques étapes de formatage nécessaires que nous allons examiner ici.


Nous sommes tenus de :
1. Ajouter des jetons spéciaux au début et à la fin de chaque phrase.
2. Remplir et tronquer toutes les phrases à une longueur unique et constante.
3. Différencier explicitement les jetons réels des jetons de remplissage à l'aide du "masque d'attention".

### Jetons spéciaux

**[SEP]`**

À la fin de chaque phrase, nous devons ajouter le jeton spécial `[SEP]`.

Ce jeton est un artefact des tâches à deux phrases, où BERT reçoit deux phrases distinctes et doit déterminer quelque chose (par exemple, la réponse à la question de la phrase A peut-elle être trouvée dans la phrase B ?)

**`[CLS]`**

Pour les tâches de classification, nous devons ajouter le jeton spécial `[CLS]` au début de chaque phrase.

Ce jeton a une signification particulière. BERT se compose de 12 couches de transformateurs. Chaque transformateur reçoit une liste d'encastrements de jetons et produit le même nombre d'encastrements en sortie (mais avec des valeurs de caractéristiques modifiées, bien sûr !)

![Illustration of CLS token purpose](http://www.mccormickml.com/assets/BERT/CLS_token_500x606.png)

À la sortie du transformateur final (12e), *seule la première intégration (correspondant au jeton [CLS]) est utilisée par le classificateur*.

> Le premier jeton de chaque séquence est toujours un jeton de classification spécial (`[CLS]`). L'état caché final
L'état caché final correspondant à ce jeton est utilisé comme représentation agrégée de la séquence pour les tâches de classification.
pour les tâches de classification."(from the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))



### Longueur de la phrase et masque d'attention

Les phrases de notre ensemble de données ont évidemment des longueurs variables, alors comment BERT gère-t-il cela ?

L'ORET a deux contraintes :
1. Toutes les phrases doivent être complétées ou tronquées à une longueur unique et fixe.
2. La longueur maximale des phrases est de 512 tokens.

Le remplissage est effectué avec un jeton spécial `[PAD]`, qui est à l'index 0 dans le vocabulaire de l'ORET. L'illustration ci-dessous montre un remplissage jusqu'à un "MAX_LEN" de 8 tokens.

<img src="http://www.mccormickml.com/assets/BERT/padding_and_mask.png" width="600">

Le "Attention Mask" est simplement un tableau de 1 et de 0 indiquant quels tokens sont padding et lesquels ne le sont pas.







### Sentences to IDs

La fonction `tokenizer.encode` combine plusieurs étapes pour nous :
1. Diviser la phrase en tokens.
2. Ajouter les jetons spéciaux `[CLS]` et `[SEP]`.
3. Associer les tokens à leurs identifiants.

Curieusement, cette fonction peut effectuer la troncature pour nous, mais ne gère pas le remplissage.

In [10]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Pour chaque phrase...
for sent in sentences:
    # `encode` va :
    # (1) Tokeniser la phrase.
    # (2) Ajouter le jeton `[CLS]` au début de la phrase.
    # (3) Ajouter le jeton `[SEP]` à la fin.
    # (4) Associer les jetons à leurs identifiants.
    encoded_sent = tokenizer.encode(
                        sent,                      # Phrase à encoder.
                        add_special_tokens = True, # Ajouter "[CLS]" et "[SEP]".

                        # Cette fonction prend également en charge la troncature et la conversion
                        # en tenseurs pytorch, mais nous avons besoin de faire du padding, donc nous
                        # ne pouvons pas utiliser ces fonctionnalités :( .
                        #max_length = 128, # Tronquer toutes les phrases.
                        #return_tensors = 'pt', # Retourne les tenseurs de pytorch.
                   )

    # Ajouter la phrase encodée à la liste.
    input_ids.append(encoded_sent)

# Imprimer la phrase 0, maintenant sous forme de liste d'ID.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


## Remplissage et troncature

Remplir et tronquer nos séquences pour qu'elles aient toutes la même longueur, `MAX_LEN`.

Tout d'abord, quelle est la longueur maximale des phrases dans notre ensemble de données ?

In [11]:
print('Durée maximale de la phrase: ', max([len(sen) for sen in input_ids]))

Durée maximale de la phrase:  47


**Compte tenu de cela, choisissons MAX_LEN = 64 et appliquons le padding.

In [12]:
# Nous allons emprunter la fonction utilitaire `pad_sequences` pour faire cela.
from keras.preprocessing.sequence import pad_sequences

# Fixer la longueur maximale de la séquence.
# J'ai choisi 64 un peu arbitrairement. C'est légèrement plus grand que la
# longueur maximale des phrases d'entraînement de 47...
MAX_LEN = 64

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Remplir nos jetons d'entrée avec la valeur 0.
# "post" indique que nous voulons remplir et tronquer à la fin de la séquence,
# plutôt qu'au début.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.


##  Masques d'attention

Le masque d'attention rend simplement explicite les jetons qui sont des mots réels par rapport à ceux qui sont du remplissage.

Le vocabulaire de l'ORET n'utilise pas l'ID 0, donc si l'ID d'un jeton est 0, il s'agit d'un remplissage, sinon c'est un vrai jeton.

In [13]:
# Créer des masques d'attention
attention_masks = []

# Pour chaque phrase...
for sent in input_ids:

    # Créer le masque d'attention.
    # - Si l'ID d'un jeton est 0, alors il s'agit d'un padding, mettez le masque à 0.
    # - Si l'ID d'un jeton est > 0, il s'agit d'un vrai jeton, mettre le masque à 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Stocker le masque d'attention pour cette phrase.
    attention_masks.append(att_mask)

## Formation & validation Split

Divisez notre ensemble de formation de manière à utiliser 90 % pour la formation et 10 % pour la validation.

In [14]:
# Utiliser train_test_split pour diviser nos données en ensembles de formation et de validation pour
# l'entraînement
from sklearn.model_selection import train_test_split

# Utilisez 90 % pour la formation et 10 % pour la validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.1)
# Faire de même pour les masques.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

## Conversion vers les types de données PyTorch

Notre modèle attend des tenseurs PyTorch plutôt que des numpy.ndarrays, il faut donc convertir toutes les variables de notre jeu de données.

In [16]:
import torch
# Convertir toutes les entrées et les étiquettes en tenseurs torch, le type de données requis
# pour notre modèle.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

Nous allons également créer un itérateur pour notre ensemble de données en utilisant la classe torch DataLoader. Cela permet d'économiser de la mémoire pendant l'apprentissage car, contrairement à une boucle for, avec un itérateur, l'ensemble des données n'a pas besoin d'être chargé en mémoire.

In [17]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Le DataLoader a besoin de connaître la taille de notre lot pour l'entraînement, nous la spécifions donc
# ici.
# Pour affiner le BERT sur une tâche spécifique, les auteurs recommandent une taille de lot de
# 16 ou 32.

batch_size = 32

# Créer le DataLoader pour notre ensemble d'entraînement.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Créer le DataLoader pour notre jeu de validation.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Entraîner notre modèle de classification

Maintenant que nos données d'entrée sont correctement formatées, il est temps d'affiner le modèle BERT.

##  BertForSequenceClassification

Pour cette tâche, nous voulons d'abord modifier le modèle BERT pré-entraîné pour obtenir des résultats pour la classification, puis nous voulons continuer à entraîner le modèle sur notre ensemble de données jusqu'à ce que le modèle entier, de bout en bout, soit bien adapté à notre tâche.

Heureusement, l'implémentation de huggingface pytorch inclut un ensemble d'interfaces conçues pour une variété de tâches NLP. Bien que ces interfaces soient toutes construites sur un modèle BERT entraîné, chacune d'entre elles possède des couches supérieures et des types de sortie différents, conçus pour s'adapter à leur tâche de TAL spécifique.  

Voici la liste actuelle des classes fournies pour un réglage fin :
* BertModel
* BertForPreTraining
* BertForMaskedLM
* BertForNextSentencePrediction (prédiction de la phrase suivante)
* **BertForSequenceClassification** - Celle que nous utiliserons.
* BertForTokenClassification
* BertForQuestionAnswering (pour les réponses aux questions)

La documentation relative à ces éléments est disponible à l'adresse suivante : [here](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Nous utiliserons [BertForSequenceClassification] (https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification). Il s'agit du modèle BERT normal auquel a été ajoutée une couche linéaire unique pour la classification que nous utiliserons comme classificateur de phrases. Au fur et à mesure que nous fournissons des données d'entrée, l'ensemble du modèle BERT pré-entraîné et la couche de classification supplémentaire non entraînée sont entraînés sur notre tâche spécifique.

In [18]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Charger BertForSequenceClassification, le modèle BERT pré-entraîné avec une seule couche de
# couche de classification linéaire.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Utiliser le modèle BERT à 12 couches, avec un vocabulaire sans majuscules.
    num_labels = 2, # Le nombre d'étiquettes de sortie - 2 pour la classification binaire.
                    # Vous pouvez augmenter cette valeur pour les tâches multi-classes.
    output_attentions = False, # Si le modèle renvoie des poids d'attention.
    output_hidden_states = False, # Si le modèle renvoie tous les états cachés.
)

# Indiquer à pytorch d'exécuter ce modèle sur le GPU.
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

Par curiosité, nous pouvons parcourir ici tous les paramètres du modèle par leur nom.

Dans la cellule ci-dessous, j'ai imprimé les noms et les dimensions des poids pour :

1. La couche d'intégration.
2. Le premier des douze transformateurs.
3. La couche de sortie.

In [19]:
# Obtenir tous les paramètres du modèle sous la forme d'une liste de tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## Optimiseur et planificateur de taux d'apprentissage

Maintenant que notre modèle est chargé, nous devons saisir les hyperparamètres d'entraînement à partir du modèle stocké.

Pour les besoins d'un réglage fin, les auteurs recommandent de choisir parmi les valeurs suivantes :
- Taille du lot : 16, 32 (nous avons choisi 32 lors de la création de nos DataLoaders).
- Taux d'apprentissage (Adam) : 5e-5, 3e-5, 2e-5 (nous utiliserons 2e-5).
- Nombre d'époques : 2, 3, 4 (nous utiliserons 4).



In [20]:
# Note : AdamW est une classe de la bibliothèque huggingface (par opposition à pytorch)
# Je crois que le 'W' signifie 'Weight Decay fix' (fixation de la décroissance du poids)
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - la valeur par défaut est 5e-5, notre notebook avait 2e-5
                  eps = 1e-8 # args.adam_epsilon - la valeur par défaut est 1e-8.
                )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
from transformers import get_linear_schedule_with_warmup
# Nombre d'époques d'apprentissage (les auteurs recommandent entre 2 et 4)
epochs = 10

# Le nombre total d'étapes d'apprentissage est égal au nombre de lots * nombre d'époques.
total_steps = len(train_dataloader) * epochs

# Créer le planificateur de taux d'apprentissage.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## Boucle de formation

Voici notre boucle d'entraînement. Il se passe beaucoup de choses, mais fondamentalement, pour chaque passage de notre boucle, nous avons une phase de formation et une phase de validation. À chaque passage, nous devons :

Boucle d'apprentissage :
- Décompresser nos données d'entrée et nos étiquettes
- Charger les données sur le GPU pour l'accélération
- Effacer les gradients calculés lors de la passe précédente.
    - Dans pytorch, les gradients s'accumulent par défaut (utile pour les RNN) à moins que vous ne les effaciez explicitement.
- Passe avant (introduire les données d'entrée dans le réseau)
- Passe arrière (rétropropagation)
- Demander au réseau de mettre à jour les paramètres avec optimizer.step()
- Suivi des variables pour contrôler la progression

Boucle d'évaluation :
- Décompressez nos données d'entrée et nos étiquettes
- Chargement des données sur le GPU pour l'accélération
- Passage en avant (alimentation des données d'entrée à travers le réseau)
- Calcul de la perte sur nos données de validation et suivi des variables pour contrôler les progrès.

Définir une fonction d'aide pour le calcul de la précision.

In [22]:
import numpy as np

# Fonction permettant de calculer la précision de nos prédictions par rapport aux étiquettes
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Fonction d'aide pour le formatage des temps écoulés.

In [23]:
import time
import datetime

def format_time(elapsed):
    '''
    Prend un temps en secondes et retourne une chaîne hh:mm:ss
    '''
    # Arrondi à la seconde la plus proche.
    elapsed_rounded = int(round((elapsed)))

    # Format hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


Nous sommes prêts à donner le coup d'envoi de la formation !

In [25]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Fixer la valeur de la graine un peu partout pour que ce soit reproductible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda")

# Stocke la perte moyenne après chaque époque afin de pouvoir la représenter graphiquement.
loss_values = []

# Pour chaque époque...
for epoch_i in range(0, epochs):

    # ========================================
    # Formation
    # ========================================

    # Effectuer un passage complet sur l'ensemble d'apprentissage.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Mesure la durée de l'époque d'apprentissage.
    t0 = time.time()

    # Réinitialiser la perte totale pour cette époque.
    total_loss = 0

    # Mettre le modèle en mode d'entraînement. Ne vous laissez pas induire en erreur - l'appel à
    # `train` ne fait que changer le *mode*, il ne *permet pas* l'apprentissage.
    # Les couches `dropout` et `batchnorm` se comportent différemment pendant l'entraînement
    # vs. test (source : https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # Pour chaque lot de données d'apprentissage...
    for step, batch in enumerate(train_dataloader):

        # Mise à jour de l'état d'avancement tous les 40 lots.
        if step % 40 == 0 and not step == 0:
            # Calculer le temps écoulé en minutes.
            elapsed = format_time(time.time() - t0)

            # Rapport sur l'état d'avancement.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Décompressez ce lot de formation à partir de notre dataloader.
        #
        # En décompressant le lot, nous copierons également chaque tenseur sur le GPU en utilisant la méthode
        # méthode `to`.
        #
        # `batch` contient trois tenseurs pytorch :
        # [0] : ids d'entrée
        # [1] : masques d'attention
        # [2] : étiquettes
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Il faut toujours effacer les gradients calculés précédemment avant d'effectuer une
        # avant d'effectuer une passe arrière. PyTorch ne le fait pas automatiquement parce que
        # l'accumulation des gradients est "pratique lors de l'entraînement des RNN".
        # (source : https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Effectuer une passe avant (évaluer le modèle sur ce lot d'entraînement).
        # Ceci retournera la perte (plutôt que la sortie du modèle) parce que nous
        # avons fourni les `étiquettes`.
        # La documentation pour cette fonction `model` est ici :
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        # L'appel à `model` renvoie toujours un tuple, donc nous devons extraire la
        # valeur de la perte du tuple.
        loss = outputs[0]

        # Accumuler la perte d'entraînement sur tous les lots afin de pouvoir
        # calculer la perte moyenne à la fin. `loss` est un tenseur contenant une seule valeur.
        # une seule valeur ; la fonction `.item()` renvoie simplement la valeur Python
        # du tenseur.
        total_loss += loss.item()

        # Effectuer une passe arrière pour calculer les gradients.
        loss.backward()

        # Réduire la norme des gradients à 1,0.
        # Cela permet d'éviter le problème des "gradients qui explosent".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Mettre à jour les paramètres et faire un pas en utilisant le gradient calculé.
        # L'optimiseur dicte la "règle de mise à jour", c'est-à-dire la manière dont les paramètres sont modifiés en fonction de leurs gradients, du taux d'apprentissage, etc.
        # modifiés en fonction de leurs gradients, du taux d'apprentissage, etc.
        optimizer.step()

        # Mettre à jour le taux d'apprentissage.
        scheduler.step()

    # Calculer la perte moyenne sur les données d'apprentissage.
    avg_train_loss = total_loss / len(train_dataloader)

    # Stocker la valeur de la perte pour tracer la courbe d'apprentissage.
    loss_values.append(avg_train_loss)

    print("")
    print("  Perte moyenne de formation: {0:.2f}".format(avg_train_loss))
    print("  Temps de formation epcoh: {:}".format(format_time(time.time() - t0)))

    # ========================================
    # Validation
    # ========================================
    # Après l'achèvement de chaque période d'apprentissage, nous mesurons notre performance sur
    # notre ensemble de validation.

    print("")
    print("Validation en cours d'exécution...")

    t0 = time.time()

    # Put the model in evaluation mode - exclusion layers behave differently
    # during evaluation.
    model.eval()

    #Suivi des variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Évaluer les données pour une époque
    for batch in validation_dataloader:

        # Ajouter le lot au GPU
        batch = tuple(t.to(device) for t in batch)

        # Décompresse les entrées de notre dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Indiquer au modèle de ne pas calculer ou stocker les gradients, ce qui permet d'économiser de la mémoire et d'accélérer la validation.
        # accélérer la validation
        with torch.no_grad():

            # Passe en avant, calcule les prédictions logit.
            # Ceci renverra les logits plutôt que les pertes car nous n'avons pas
            # n'avons pas fourni d'étiquettes.
            # token_type_ids est le même que le "segment ids", qui # différencie la phrase 1 et 2 dans les tâches à 2 phrases.
            # différencie les phrases 1 et 2 dans les tâches à 2 phrases.
            # La documentation pour cette fonction `modèle` est ici :
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Obtenir les "logits" produits par le modèle. Les "logits" sont les valeurs de sortie
        # avant l'application d'une fonction d'activation comme la softmax.
        logits = outputs[0]

        # Déplacer les logits et les étiquettes vers l'unité centrale
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculer la précision pour ce lot de phrases de test.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumuler la précision totale.
        eval_accuracy += tmp_eval_accuracy

        # Suivre le nombre de lots
        nb_eval_steps += 1

    # Indique la précision finale pour ce cycle de validation.
    print("  Précision: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Formation terminée!")


======== Epoch 1 / 10 ========
Training...
  Batch    40  of    241.    Elapsed: 0:00:13.
  Batch    80  of    241.    Elapsed: 0:00:25.
  Batch   120  of    241.    Elapsed: 0:00:37.
  Batch   160  of    241.    Elapsed: 0:00:49.
  Batch   200  of    241.    Elapsed: 0:01:02.
  Batch   240  of    241.    Elapsed: 0:01:15.

  Perte moyenne de formation: 0.49
  Temps de formation epcoh: 0:01:15

Validation en cours d'exécution...
  Précision: 0.81
  Validation took: 0:00:03

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    241.    Elapsed: 0:00:13.
  Batch    80  of    241.    Elapsed: 0:00:25.
  Batch   120  of    241.    Elapsed: 0:00:38.
  Batch   160  of    241.    Elapsed: 0:00:51.
  Batch   200  of    241.    Elapsed: 0:01:04.
  Batch   240  of    241.    Elapsed: 0:01:17.

  Perte moyenne de formation: 0.30
  Temps de formation epcoh: 0:01:17

Validation en cours d'exécution...
  Précision: 0.81
  Validation took: 0:00:03

======== Epoch 3 / 10 ========
Training..